In [0]:
from pyspark.sql import SparkSession

spark=SparkSession.builder.appName("PSL Data Analysis").getOrCreate()

In [0]:
spark

In [0]:

dbutils.fs.mount(
    source="wasbs://container@storage.blob.core.windows.net",
    mount_point='/mnt/psl-data',
    extra_configs={'fs.azure.account.key.pslstorageac.blob.core.windows.net':dbutils.secrets.get('scope', 'key')
    }
)

In [0]:
from pyspark.sql.types import StructField,StructType,IntegerType,StringType, BooleanType,DecimalType,DataType
from pyspark.sql.functions import col,sum,avg,min,max,count,when
from pyspark.sql.window import Window


In [0]:
%fs
ls /mnt/psl-data

path,name,size,modificationTime
dbfs:/mnt/psl-data/PSL-2016-2022 DataSet.csv,PSL-2016-2022 DataSet.csv,4104341,1714133783000


In [0]:
schema_ball_by_ball= StructType([
    StructField("psl_year",IntegerType(),True),
    StructField("match_number",IntegerType(),True),
    StructField("team_1",StringType(),True),
    StructField("team_2",StringType(),True),
    StructField("inning",IntegerType(),True),
    StructField("over",IntegerType(),True),
    StructField("ball",IntegerType(),True),
    StructField("runs",IntegerType(),True),
    StructField("wicket",IntegerType(),True),
    StructField("total_runs",IntegerType(),True),
    StructField("wickets",IntegerType(),True),
    StructField("is_four",BooleanType(),True),
    StructField("is_six",BooleanType(),True),
    StructField("is_wicket",BooleanType(),True),
    StructField("wicket_text",StringType(),True),
    StructField("result",StringType(),True)
])

In [0]:
ball_by_ball=spark.read.csv('dbfs:/mnt/psl-data/PSL-2016-2022 DataSet.csv', header=True,schema=schema_ball_by_ball)

In [0]:
ball_by_ball.printSchema()

root
 |-- psl_year: integer (nullable = true)
 |-- match_number: integer (nullable = true)
 |-- team_1: string (nullable = true)
 |-- team_2: string (nullable = true)
 |-- inning: integer (nullable = true)
 |-- over: integer (nullable = true)
 |-- ball: integer (nullable = true)
 |-- runs: integer (nullable = true)
 |-- wicket: integer (nullable = true)
 |-- total_runs: integer (nullable = true)
 |-- wickets: integer (nullable = true)
 |-- is_four: boolean (nullable = true)
 |-- is_six: boolean (nullable = true)
 |-- is_wicket: boolean (nullable = true)
 |-- wicket_text: string (nullable = true)
 |-- result: string (nullable = true)



In [0]:
ball_by_ball.show(5)

+--------+------------+----------------+-----------------+------+----+----+----+------+----------+-------+-------+------+---------+-----------+----------+
|psl_year|match_number|          team_1|           team_2|inning|over|ball|runs|wicket|total_runs|wickets|is_four|is_six|is_wicket|wicket_text|    result|
+--------+------------+----------------+-----------------+------+----+----+----+------+----------+-------+-------+------+---------+-----------+----------+
|    2016|           1|Islamabad United|Quetta Gladiators|     1|   1|   1|   0|  NULL|         0|      0|  false| false|    false|       NULL|Gladiators|
|    2016|           1|Islamabad United|Quetta Gladiators|     1|   1|   2|   0|  NULL|         0|      0|  false| false|    false|       NULL|Gladiators|
|    2016|           1|Islamabad United|Quetta Gladiators|     1|   1|   3|   0|  NULL|         0|      0|  false| false|    false|       NULL|Gladiators|
|    2016|           1|Islamabad United|Quetta Gladiators|     1|   1|

In [0]:
match_info = ball_by_ball.select("psl_year", "match_number", "team_1","ball", "team_2", "result") \
            .groupBy("match_number", "psl_year", "team_1", "team_2", "result") \
            .agg(count("ball").alias('total_ball')).sort('psl_year','match_number')

In [0]:
match_info.show(5)

+------------+--------+----------------+-----------------+----------+----------+
|match_number|psl_year|          team_1|           team_2|    result|total_ball|
+------------+--------+----------------+-----------------+----------+----------+
|           1|    2016|Islamabad United|Quetta Gladiators|Gladiators|       223|
|           2|    2016|Lahore Qalandars|    Karachi Kings|     Kings|       223|
|           3|    2016|  Peshawar Zalmi| Islamabad United|     Zalmi|       251|
|           4|    2016|   Karachi Kings|Quetta Gladiators|Gladiators|       235|
|           5|    2016|Lahore Qalandars|   Peshawar Zalmi|     Zalmi|       226|
+------------+--------+----------------+-----------------+----------+----------+
only showing top 5 rows



In [0]:
# I want to add unique id to all the unique teams in team1
teams = match_info.selectExpr("DENSE_RANK() OVER (ORDER BY team_1) as team_id", "team_1").distinct()

In [0]:
teams.show()

+-------+-----------------+
|team_id|           team_1|
+-------+-----------------+
|      1| Islamabad United|
|      2|    Karachi Kings|
|      3| Lahore Qalandars|
|      4|   Multan Sultans|
|      5|   Peshawar Zalmi|
|      6|Quetta Gladiators|
+-------+-----------------+



In [0]:
# Calculate the average and total runs by inning and match
avg_runs_by_inning_match = ball_by_ball.groupBy("psl_year","match_number","inning" ).agg(avg("runs").alias("avg_runs"), sum("runs").alias("total_runs")).sort("psl_year","match_number","inning" )
avg_runs_by_inning_match.show(5)

+--------+------------+------+------------------+----------+
|psl_year|match_number|inning|          avg_runs|total_runs|
+--------+------------+------+------------------+----------+
|    2016|           1|     1| 1.032258064516129|       128|
|    2016|           1|     2|1.3333333333333333|       132|
|    2016|           2|     1|               1.0|       125|
|    2016|           2|     2| 1.336734693877551|       131|
|    2016|           3|     1|1.1507936507936507|       145|
+--------+------------+------+------------------+----------+
only showing top 5 rows



In [0]:
#Running avaerage by over 
window_fun=Window.partitionBy("psl_year","match_number","inning" ).orderBy("inning")
running_avg_by_over=ball_by_ball.withColumn('running_total_by_over',sum('runs').over(window_fun))
running_avg_by_over.show(5)


+--------+------------+----------------+-----------------+------+----+----+----+------+----------+-------+-------+------+---------+-----------+----------+---------------------+
|psl_year|match_number|          team_1|           team_2|inning|over|ball|runs|wicket|total_runs|wickets|is_four|is_six|is_wicket|wicket_text|    result|running_total_by_over|
+--------+------------+----------------+-----------------+------+----+----+----+------+----------+-------+-------+------+---------+-----------+----------+---------------------+
|    2016|           1|Islamabad United|Quetta Gladiators|     1|   1|   1|   0|  NULL|         0|      0|  false| false|    false|       NULL|Gladiators|                  128|
|    2016|           1|Islamabad United|Quetta Gladiators|     1|   1|   2|   0|  NULL|         0|      0|  false| false|    false|       NULL|Gladiators|                  128|
|    2016|           1|Islamabad United|Quetta Gladiators|     1|   1|   3|   0|  NULL|         0|      0|  false| 

In [0]:
#High Impact 
hight_impact=ball_by_ball.withColumn('hight_impact',when((col('is_six')==True) | (col('wicket')==True),True).otherwise(False))
hight_impact.show(5)

+--------+------------+----------------+-----------------+------+----+----+----+------+----------+-------+-------+------+---------+-----------+----------+------------+
|psl_year|match_number|          team_1|           team_2|inning|over|ball|runs|wicket|total_runs|wickets|is_four|is_six|is_wicket|wicket_text|    result|hight_impact|
+--------+------------+----------------+-----------------+------+----+----+----+------+----------+-------+-------+------+---------+-----------+----------+------------+
|    2016|           1|Islamabad United|Quetta Gladiators|     1|   1|   1|   0|  NULL|         0|      0|  false| false|    false|       NULL|Gladiators|       false|
|    2016|           1|Islamabad United|Quetta Gladiators|     1|   1|   2|   0|  NULL|         0|      0|  false| false|    false|       NULL|Gladiators|       false|
|    2016|           1|Islamabad United|Quetta Gladiators|     1|   1|   3|   0|  NULL|         0|      0|  false| false|    false|       NULL|Gladiators|      

In [0]:
#Runs by innings 

runs_by_innings= ball_by_ball.groupBy("psl_year","match_number","inning", ).agg(sum(col('runs')).alias('total_runs')).sort("psl_year","match_number","inning","total_runs")
runs_by_innings.show(5)

+--------+------------+------+----------+
|psl_year|match_number|inning|total_runs|
+--------+------------+------+----------+
|    2016|           1|     1|       128|
|    2016|           1|     2|       132|
|    2016|           2|     1|       125|
|    2016|           2|     2|       131|
|    2016|           3|     1|       145|
+--------+------------+------+----------+
only showing top 5 rows



In [0]:
#most matches won by team

hight_impact.createOrReplaceTempView("ball_by_ball")

In [0]:
#Total no of machtes won by each team

spark.sql("""
SELECT result, count(result) AS No_of_maches_won
FROM(SELECT result,count(result) AS count 
FROM ball_by_ball
WHERE inning = 1
GROUP BY  psl_year,match_number,inning,result  
ORDER BY  psl_year,match_number,inning)
GROUP BY (result)
ORDER BY No_of_maches_won DESC
      
""").display()

result,No_of_maches_won
Zalmi,43
United,41
Gladiators,36
Sultans,30
Qalandars,30
Kings,29
tied,4
no_result,2


In [0]:
#Total no of sixes in each inning

spark.sql("""
SELECT psl_year,match_number,inning, count(is_six) AS No_of_six 
FROM ball_by_ball
WHERE is_six=TRUE
GROUP BY  psl_year,match_number,inning  
ORDER BY psl_year,match_number,inning, No_of_six DESC
""").display()

psl_year,match_number,inning,No_of_six
2016,1,1,4
2016,1,2,4
2016,2,1,2
2016,2,2,7
2016,3,1,3
2016,3,2,2
2016,4,1,3
2016,4,2,8
2016,5,1,1
2016,5,2,3


In [0]:
#Total no of four in each inning

spark.sql("""
SELECT psl_year,match_number,inning, count(is_four) AS No_of_four 
FROM ball_by_ball
WHERE is_four=TRUE
GROUP BY  psl_year,match_number,inning  
ORDER BY psl_year,match_number,inning, No_of_four DESC
""").display()

psl_year,match_number,inning,No_of_four
2016,1,1,10
2016,1,2,14
2016,2,1,9
2016,2,2,11
2016,3,1,10
2016,3,2,10
2016,4,1,9
2016,4,2,10
2016,5,1,11
2016,5,2,10


In [0]:
#Total No of boundries each Inning 
spark.sql("""
SELECT psl_year,match_number,inning, ((count(is_four)+count(is_six))/2) AS No_of_boundries 
FROM ball_by_ball
WHERE is_four=TRUE OR is_six=TRUE
GROUP BY  psl_year,match_number,inning  
ORDER BY psl_year,match_number,inning, No_of_boundries DESC
""").display()

psl_year,match_number,inning,No_of_boundries
2016,1,1,14.0
2016,1,2,18.0
2016,2,1,11.0
2016,2,2,18.0
2016,3,1,13.0
2016,3,2,12.0
2016,4,1,12.0
2016,4,2,18.0
2016,5,1,12.0
2016,5,2,13.0


In [0]:
#Total no of wickets in each inning

No_of_wicket_by_inning=spark.sql("""
SELECT psl_year,match_number,inning, count(is_wicket) AS No_of_wicket 
FROM ball_by_ball
WHERE is_wicket=TRUE
GROUP BY  psl_year,match_number,inning  
ORDER BY psl_year,match_number,inning, No_of_wicket DESC
""")
No_of_wicket_by_inning.display()

psl_year,match_number,inning,No_of_wicket
2016,1,1,7
2016,1,2,2
2016,2,1,8
2016,2,2,3
2016,3,1,7
2016,3,2,9
2016,4,1,7
2016,4,2,2
2016,5,1,6
2016,5,2,1


In [0]:
#Total no of wickets in each year

spark.sql("""
SELECT psl_year , count(is_wicket) as total_wickets 
FROM ball_by_ball
WHERE is_wicket=TRUE
GROUP BY psl_year
ORDER BY psl_year
""").display()


psl_year,total_wickets
2016,283
2017,313
2018,415
2019,432
2020,373
2021,417
2022,428


In [0]:
#Total no of sixes in each year
spark.sql("""
SELECT psl_year , count(is_six) as total_six 
FROM ball_by_ball
WHERE is_six=TRUE
GROUP BY psl_year
ORDER BY psl_year
""").display()

psl_year,total_six
2016,247
2017,258
2018,414
2019,377
2020,383
2021,437
2022,435
